# Fixed and Variable Length Codes
In the previous lecture, we showed how to exploit redundancy by building a lookup table that mapped one of 4 possible strings to a 2-bit key:

| Color | key |
|-------|-------|
| Red   | 00 |
| Green  | 01|
| Blue  | 10|
| Black  | 11|

Such codes are called "fixed-length" codes because the length of the key (in bits) is a pre-determined size. Let's overview some of the advantages of these codes with an example.

First, we are going to define two data structures that represent this table above:

In [40]:
encode_table = {'red': (0,0), 'green': (0,1), 'blue': (1,0), 'black': (1,1) }

decode_table = {(0,0):'red', (0,1): 'green', (1,0): 'blue', (1,1):'black'}

print('The code for black is: ',encode_table['blue'])
print('The symbol for 0,1 is: ', decode_table[(0,1)])
print('The get the symbol back: ',decode_table[encode_table['black']])

The code for black is:  (1, 0)
The symbol for 0,1 is:  green
The get the symbol back:  black


Now, we can write an encoder and decoder function that takes a list of such colors and produces a code string and vice versa:

In [44]:
def encode_fixed(lst, table, code_length=2):
    #lst: is a list of symbols such as ['red', 'green', 'blue']
    #table: an encoding table
    #code_length: length of the codes in the table
    
    #return a new list that contains the encoded list
    
    
    output = [None]*(code_length*len(lst)) #create an empty list with the appropriate size
    
    
    for i,sym in enumerate(lst):
        code = table[sym]
        output[i*code_length:(i+1)*(code_length)] = code

    return output


print(encode_fixed(['red','red','green','black'], encode_table))
print(encode_fixed(['red','blue','red','black'], encode_table))
print(encode_fixed(['red','blue','red','black'], encode_table, code_length=3))

[0, 0, 0, 0, 0, 1, 1, 1]
[0, 0, 1, 0, 0, 0, 1, 1]
[0, 0, None, 1, 0, None, 0, 0, None, 1, 1]


In [46]:
def decode_fixed(enc, table, code_length=2):
    #enc list e.g., [0, 0, 0, 0, 0, 1, 1, 1]
    #table: a table mapping code words to symbols
    #code_length
    
    output = [None]*(len(enc) // code_length) #create an empty list with the appropriate size
    
    for i in range(0, len(enc), code_length):
        sym = table[tuple(enc[i:i+code_length])]
        output[i//code_length] = sym

    return output


print(decode_fixed([0,0,0,1,1,1], decode_table))

enc = encode_fixed(['red','red','green','black'], encode_table)
dec = decode_fixed(enc, decode_table)

print(dec)

['red', 'green', 'black']
['red', 'red', 'green', 'black']


That's essentially fixed length encoding, but the key issue is that this scheme is a bit wasteful because we allocate the same number of bits to the most popular colors as the least. Suppose, we new that almost all of the colors in our list were Red. Is there a way to dynamically adjust our encoding to use less space to store strings that we know will show up more often?

## A Naive Approach

Let's try manually adjust our encoding to scale with popularity. Consider the lookup table before (with the strings annotated with popularity). We sort the strings from most to least popular, assign them a binary key, and then prune the leading zeros.

| Color | key |
|-------|-------|
| Red (0.8)   | 0 |
| Green (0.02)  |11|
| Blue (0.03)  | 10|
| Black (0.15)  | 1|


Our previous dictionary encoding approach requires 2-bits per string on average. Let's calculate the average (or expected) storage cost for this new encoding:

$$0.8*1 + 0.15*1 + 0.02 * 2 + 0.03 * 2 = 1.05$$

Such an encoding is called a "variable-length" encoding as it varies with the symbol. Let's see how this works in code

In [48]:
encode_table = {'red': (0,), 'green': (1,1), 'blue': (1,0), 'black': (1,) }

def encode_variable(lst, table):
    
    output = [] #hard to guess the size before hand have to dynamically size
    
    for sym in lst:
        code = table[sym]
        output.extend(code)

    return output

encode_variable(['red','red','green','black'], encode_table)

[0, 0, 1, 1, 1]

This means that simply changing the encoding compresses the data by about 50%. But, what goes wrong with this approach? Suppose, we retrieved the bits 10--how do we decode that? Is it "Black, Red" or is it "Blue". This is the core issue with this naive approach that there is amiguity in the decoding.

In [49]:
print(encode_variable(['black','red'], encode_table))
print(encode_variable(['blue'], encode_table))

[1, 0]
[1, 0]


## Prefix-Free Codes
A careful reader might note that we could side-step this problem (and still get a compression benefit) by encoding the data slightly differently:

| Color | key |
|-------|-------|
| Red (0.8)   | 0 |
| Green (0.02)  |110|
| Blue (0.03)  | 111|
| Black (0.15)  | 10|

$$0.8*1 + 0.15*2 + 0.03 * 3 + 0.02 * 3 = 1.25$$

If read the data left to right, there is no longer any ambiguity. We simply scan the list until we find a "full" key in our lookup table. 

In [51]:
def decode_variable(enc, table):
    
    output = [] #hard to guess the size before hand have to dynamically size
    
    buffer = [] #buffer up because you don't know when the current word will end
    
    for bit in enc:
        buffer.append(bit)
        
        if tuple(buffer) in table:
            output.append(table[tuple(buffer)])
            buffer = []
            
    
    if len(buffer) > 0:
        output.append(table[tuple(buffer)]) #why do we need this???

    return output

encode_table = {'red': (0,), 'green': (1,1,0), 'blue': (1,1,1), 'black': (1,0) }
decode_table = {(0,):'red', (1,1,0):'green', (1,1,1):'blue', (1,0):'black' }

enc1 = encode_variable(['black','red', 'green'], encode_table)
print(enc1)

enc2 = encode_variable(['blue', 'black'], encode_table)
print(enc2)

print(decode_variable(enc1, decode_table))
print(decode_variable(enc2, decode_table))
    

[1, 0, 0, 1, 1, 0]
[1, 1, 1, 1, 0]
['black', 'red', 'green']
['blue', 'black']


This works because each of the keys is prefix-free. A prefix-free encoding is a key that requires that there is no whole code word in the system that is a prefix (initial segment) of any other code word in the system. Luckily, we don't have to design such schemes by hand. There are algorithms that can find very efficient prefix-free encodings. 

## Huffman Coding
Huffman's encodes data by building a binary tree where the leaves are strings. The basic strategy is to go bottom up. Keep on grouping the lowest probability strings into common prefixes until there are none left:

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/74/Huffman_coding_example.svg/1920px-Huffman_coding_example.svg.png" width="500"/>

We can easily express this algorithm in code. The first step is to write the bottom up algorithm that builds the tree of symbols.

In [24]:
#expect input of the form [(.95,['s1']), (.05,['s2'])]

def build_tree(strings):
    '''Given a set of strings and their associated
       probabilities build a tree with nested lists
    '''
    
    while len(strings) > 1:
        
        strings.sort(key=lambda x: x[0]) #sort by 
        
        p1,s1 = strings[0] #get lowest
        p2,s2 = strings[1] #get lowest
        
        del strings[0]
        del strings[0]
        
        strings.append((p1+p2, ((p1,s1),(p2,s2)) ))
        
        #print(strings)
    
    return strings[0]

#tree = build_tree([(.65,['red']), (.02,['green']), (.03,['blue']), (.15,['black']), (.15,['magenta'])])
tree = build_tree([( (6-abs(7-roll))/36 ,[str(roll)]) for roll in range(2,13,1)])

print(tree)
  

(1.0, ((0.41666666666666663, ((0.19444444444444442, ((0.08333333333333333, ['10']), (0.1111111111111111, ['5']))), (0.2222222222222222, ((0.1111111111111111, ['9']), (0.1111111111111111, ((0.05555555555555555, ['3']), (0.05555555555555555, ['11']))))))), (0.5833333333333334, ((0.2777777777777778, ((0.1388888888888889, ['6']), (0.1388888888888889, ['8']))), (0.3055555555555556, ((0.1388888888888889, ((0.05555555555555555, ((0.027777777777777776, ['2']), (0.027777777777777776, ['12']))), (0.08333333333333333, ['4']))), (0.16666666666666666, ['7'])))))))


Then, we have a top down traversal that assigns codes:

In [25]:
def assign_codes(tree, code=''):
    
    p,s = tree
    
    if len(s) == 1:
       print(s, code)
    else:
        assign_codes(s[0],code+'1')
        assign_codes(s[1],code+'0')

assign_codes(tree)

['10'] 111
['5'] 110
['9'] 101
['3'] 1001
['11'] 1000
['6'] 011
['8'] 010
['2'] 00111
['12'] 00110
['4'] 0010
['7'] 000
